# Bert-based morphological tagger and disambiguator (BertMorphTagger)

EstNLTK contains a Bert-based morphological tagger for annotating partofpseech and morphological form features in Estonian texts using [Vabamorf's tagset](00_tables_of_morphological_categories.ipynb). 
The tagger can also be used as a disambiguator to resolve ambiguities of an existing morphological analysis layer that uses Vabamorf's tagset. 

## Prerequisites

*Note: you need to install [estnltk_neural](https://github.com/estnltk/estnltk/tree/main/estnltk_neural) and [sentencepiece](https://pypi.org/project/sentencepiece/) packages for using the Bert-based morphological tagger.*

Note that the Bert model required by the tagger is not distributed with the estnltk\_neural package. You can download the model in the following ways:
* If you create a new instance of `BertMorphTagger` and the model has not been downloaded yet, you'll be prompted with a question asking for a permission to download the model;
* Alternatively, you can pre-download the model manually via the download function:

```python
from estnltk import download
download('bert_morph_tagging')
```

Import modules

To use `BertMorphTagger`, you need to import BertMorphTagger from `estnltk_neural.taggers`:

In [1]:
import estnltk
from estnltk_neural.taggers import BertMorphTagger

### Using BertMorphTagger as a tagger

To initialize BertMorphTagger, call `BertMorphTagger` with preferred parameter values.

Important parameters for the tagger are:
* `get_top_n_predictions`: Output $n$ most probable morphological tags for each word. Defaults to **1**;
* `token_level`: If True outputs morphological tags for each BERT token, otherwise for each word in EstNLTK's `words` layer. Defaults to **False**;
* `split_pos_form`: If True splits predicted label into Vabamorf's tags `form` and `partofspeech`, otherwise outputs the label itself, which is a concatenation of previous Vabamorf's tags joined with `_`. Defaults to **True**

In [25]:
# morph_tagger = BertMorphTagger(get_top_n_predictions=1, token_level=False, split_pos_form=True) # With defined parameters
morph_tagger = BertMorphTagger() # Without defined parameters

Fetching 11 files: 100%|██████████| 11/11 [00:21<00:00,  1.94s/it]


`BertMorphTagger` relies on EstNLTK's `sentences` and `words` layer. Without it, tagger will not work.

In [26]:
input_str = "A. H. Tammsaare oli eesti kirjanik, esseist, kultuurifilosoof ja tõlkija."
text_obj = estnltk.Text(input_str)
# text_obj.tag_layer('sentences') # Without sentences and words layer
try:
    morph_tagger.tag(text_obj)
except ValueError as e:
    print(f"ValueError: {e}")

ValueError: missing input layer: 'sentences'


In [27]:
input_str = "A. H. Tammsaare oli eesti kirjanik, esseist, kultuurifilosoof ja tõlkija."
text_obj = estnltk.Text(input_str)
text_obj.tag_layer('sentences') # With sentences and words layer
morph_tagger.tag(text_obj)

Text(text='A. H. Tammsaare oli eesti kirjanik, esseist, kultuurifilosoof ja tõlkija.')

Accessing `bert_morph_tagging` layer.

In [28]:
text_obj['bert_morph_tagging']

Layer(name='bert_morph_tagging', attributes=('bert_tokens', 'form', 'partofspeech', 'probability'), spans=SL[Span('A. H. Tammsaare', [{'bert_tokens': ['▁A', '.', '▁H', '.', '▁Tammsaare'], 'form': 'sg n', 'partofspeech': 'H', 'probability': 0.99541}]),
Span('oli', [{'bert_tokens': ['▁oli'], 'form': 's', 'partofspeech': 'V', 'probability': 0.99992}]),
Span('eesti', [{'bert_tokens': ['▁eesti'], 'form': '', 'partofspeech': 'G', 'probability': 0.99577}]),
Span('kirjanik', [{'bert_tokens': ['▁kirjanik'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.9997}]),
Span(',', [{'bert_tokens': [','], 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}]),
Span('esseist', [{'bert_tokens': ['▁es', 'se', 'ist'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99956}]),
Span(',', [{'bert_tokens': [','], 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}]),
Span('kultuurifilosoof', [{'bert_tokens': ['▁kultuuri', 'filosoof'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99963}]),
Span('ja', [{'bert_tokens': ['▁ja'], 'form': '', 'partofspeech': 'J', 'probability': 0.99988}]),
Span('tõlkija', [{'bert_tokens': ['▁tõlkija'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99926}]),
Span('.', [{'bert_tokens': ['.'], 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}])])

BertMorphTagger generates four tags:
* `bert_tokens`: List of BERT tokens that are in one Vabamorf's word;
* `form`: Noun or verb form (following [Vabamorf's tagset](00_tables_of_morphological_categories.ipynb));
* `partofspeech`: Word's part of speech (following [Vabamorf's tagset](00_tables_of_morphological_categories.ipynb));
* `probability`: probability of generated tags.

Below is an example of one Vabamorf's word and its related BertMorphTagger's tokens.

In [29]:
print(f"""Vabamorf's word: {text_obj['words'][0].text}
BertMorphTagger's tokens: {text_obj['bert_morph_tagging'][0]['bert_tokens'][0]}""")

Vabamorf's word: A. H. Tammsaare
BertMorphTagger's tokens: ['▁A', '.', '▁H', '.', '▁Tammsaare']


By default, BertMorphTagger only outputs the most probable partofspeech/form tag for each word. 

However, you can use the parameter `get_top_n_predictions` to output more partofspeech/form interpretations. 

Example: let's change `get_top_n_predictions` parameter to 2 to output two top-most probability predictions for each word:

In [30]:
morph_tagger = BertMorphTagger(get_top_n_predictions=2)
input_str = "A. H. Tammsaare oli eesti kirjanik, esseist, kultuurifilosoof ja tõlkija."
text_obj = estnltk.Text(input_str)
text_obj.tag_layer('sentences') # With sentences and words layer
morph_tagger.tag(text_obj)
text_obj['bert_morph_tagging']

Layer(name='bert_morph_tagging', attributes=('bert_tokens', 'form', 'partofspeech', 'probability'), spans=SL[Span('A. H. Tammsaare', [{'bert_tokens': ['▁A', '.', '▁H', '.', '▁Tammsaare'], 'form': 'sg n', 'partofspeech': 'H', 'probability': 0.99541}, {'bert_tokens': ['▁A', '.', '▁H', '.', '▁Tammsaare'], 'form': 'sg g', 'partofspeech': 'H', 'probability': 0.00244}]),
Span('oli', [{'bert_tokens': ['▁oli'], 'form': 's', 'partofspeech': 'V', 'probability': 0.99992}, {'bert_tokens': ['▁oli'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.0}]),
Span('eesti', [{'bert_tokens': ['▁eesti'], 'form': '', 'partofspeech': 'G', 'probability': 0.99577}, {'bert_tokens': ['▁eesti'], 'form': 'sg g', 'partofspeech': 'S', 'probability': 0.00042}]),
Span('kirjanik', [{'bert_tokens': ['▁kirjanik'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.9997}, {'bert_tokens': ['▁kirjanik'], 'form': 'sg n', 'partofspeech': 'A', 'probability': 3e-05}]),
Span(',', [{'bert_tokens': [','], 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}, {'bert_tokens': [','], 'form': '', 'partofspeech': 'J', 'probability': 1e-05}]),
Span('esseist', [{'bert_tokens': ['▁es', 'se', 'ist'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99956}, {'bert_tokens': ['▁es', 'se', 'ist'], 'form': 'sg n', 'partofspeech': 'A', 'probability': 9e-05}]),
Span(',', [{'bert_tokens': [','], 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}, {'bert_tokens': [','], 'form': '', 'partofspeech': 'J', 'probability': 0.0}]),
Span('kultuurifilosoof', [{'bert_tokens': ['▁kultuuri', 'filosoof'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99963}, {'bert_tokens': ['▁kultuuri', 'filosoof'], 'form': 'sg n', 'partofspeech': 'A', 'probability': 5e-05}]),
Span('ja', [{'bert_tokens': ['▁ja'], 'form': '', 'partofspeech': 'J', 'probability': 0.99988}, {'bert_tokens': ['▁ja'], 'form': '', 'partofspeech': 'Z', 'probability': 1e-05}]),
Span('tõlkija', [{'bert_tokens': ['▁tõlkija'], 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99926}, {'bert_tokens': ['▁tõlkija'], 'form': 'sg g', 'partofspeech': 'S', 'probability': 0.00013}]),
Span('.', [{'bert_tokens': ['.'], 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}, {'bert_tokens': ['.'], 'form': '', 'partofspeech': 'J', 'probability': 0.0}])])

Parameter `token_level` can be used to switch tokenization of the output layer from the EstNLTK's words layer (the default setting) to original tokenization that was produced by the tokenizer of the Bert model. 

Example: changing `token_level` parameter to **True**:

In [31]:
morph_tagger = BertMorphTagger(token_level=True)
input_str = "A. H. Tammsaare oli eesti kirjanik, esseist, kultuurifilosoof ja tõlkija."
text_obj = estnltk.Text(input_str)
text_obj.tag_layer('sentences') # With sentences and words layer
morph_tagger.tag(text_obj)
text_obj['bert_morph_tagging']

Layer(name='bert_morph_tagging', attributes=('bert_tokens', 'form', 'partofspeech', 'probability'), spans=SL[Span('A', [{'bert_tokens': '▁A', 'form': 'sg n', 'partofspeech': 'H', 'probability': 0.99541}]),
Span('.', [{'bert_tokens': '.', 'form': '', 'partofspeech': 'Z', 'probability': 0.99949}]),
Span(' H', [{'bert_tokens': '▁H', 'form': 'sg n', 'partofspeech': 'H', 'probability': 0.99685}]),
Span('.', [{'bert_tokens': '.', 'form': '', 'partofspeech': 'Z', 'probability': 0.99929}]),
Span(' Tammsaare', [{'bert_tokens': '▁Tammsaare', 'form': 'sg n', 'partofspeech': 'H', 'probability': 0.99937}]),
Span(' oli', [{'bert_tokens': '▁oli', 'form': 's', 'partofspeech': 'V', 'probability': 0.99992}]),
Span(' eesti', [{'bert_tokens': '▁eesti', 'form': '', 'partofspeech': 'G', 'probability': 0.99577}]),
Span(' kirjanik', [{'bert_tokens': '▁kirjanik', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.9997}]),
Span(',', [{'bert_tokens': ',', 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}]),
Span(' es', [{'bert_tokens': '▁es', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99956}]),
Span('se', [{'bert_tokens': 'se', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99957}]),
Span('ist', [{'bert_tokens': 'ist', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99956}]),
Span(',', [{'bert_tokens': ',', 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}]),
Span(' kultuuri', [{'bert_tokens': '▁kultuuri', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99963}]),
Span('filosoof', [{'bert_tokens': 'filosoof', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99969}]),
Span(' ja', [{'bert_tokens': '▁ja', 'form': '', 'partofspeech': 'J', 'probability': 0.99988}]),
Span(' tõlkija', [{'bert_tokens': '▁tõlkija', 'form': 'sg n', 'partofspeech': 'S', 'probability': 0.99926}]),
Span('.', [{'bert_tokens': '.', 'form': '', 'partofspeech': 'Z', 'probability': 0.99995}])])

Parameter `split_pos_form` can be used to alternate between outputting `partofspeech` and `form` as separate features (the default setting), and outputting a single morphological tag (the attribute `morph_label`).

Example: changing `split_pos_form` parameter to **False**:

In [32]:
morph_tagger = BertMorphTagger(split_pos_form=False)
input_str = "A. H. Tammsaare oli eesti kirjanik, esseist, kultuurifilosoof ja tõlkija."
text_obj = estnltk.Text(input_str)
text_obj.tag_layer('sentences') # With sentences and words layer
morph_tagger.tag(text_obj)
text_obj['bert_morph_tagging']

Layer(name='bert_morph_tagging', attributes=('bert_tokens', 'morph_label', 'probability'), spans=SL[Span('A. H. Tammsaare', [{'bert_tokens': ['▁A', '.', '▁H', '.', '▁Tammsaare'], 'morph_label': 'sg n_H', 'probability': 0.99541}]),
Span('oli', [{'bert_tokens': ['▁oli'], 'morph_label': 's_V', 'probability': 0.99992}]),
Span('eesti', [{'bert_tokens': ['▁eesti'], 'morph_label': 'G', 'probability': 0.99577}]),
Span('kirjanik', [{'bert_tokens': ['▁kirjanik'], 'morph_label': 'sg n_S', 'probability': 0.9997}]),
Span(',', [{'bert_tokens': [','], 'morph_label': 'Z', 'probability': 0.99995}]),
Span('esseist', [{'bert_tokens': ['▁es', 'se', 'ist'], 'morph_label': 'sg n_S', 'probability': 0.99956}]),
Span(',', [{'bert_tokens': [','], 'morph_label': 'Z', 'probability': 0.99995}]),
Span('kultuurifilosoof', [{'bert_tokens': ['▁kultuuri', 'filosoof'], 'morph_label': 'sg n_S', 'probability': 0.99963}]),
Span('ja', [{'bert_tokens': ['▁ja'], 'morph_label': 'J', 'probability': 0.99988}]),
Span('tõlkija', [{'bert_tokens': ['▁tõlkija'], 'morph_label': 'sg n_S', 'probability': 0.99926}]),
Span('.', [{'bert_tokens': ['.'], 'morph_label': 'Z', 'probability': 0.99995}])])

As you can see above, tags `form` and `partofspeech` are concatenated into one tag `morph_label`.

## Using BertMorphTagger as a disambiguator

If the flag `disambiguate` is set to `True`, then the tagger can be used as an disambiguator (retagger) of an existing Vabamorf-based morph analysis layer. For this, the `output_layer` name also needs to be changed to the name of the morph analysis layer, which will become the input layer of the tagger.

Example

In [2]:
from estnltk import Text
from estnltk_neural.taggers import BertMorphTagger

In [3]:
# initialize BertMorphTagger as a disambiguator
morph_disambiguator = BertMorphTagger(output_layer='morph_analysis', disambiguate=True)

In [4]:
morph_disambiguator.input_layers

('sentences', 'words', 'morph_analysis')

In [5]:
# Use Vabamorf's analyzer to create an ambiguous morph analysis layer
from estnltk.taggers import VabamorfAnalyzer
vm_analyzer = VabamorfAnalyzer()
text = Text("Kärbes hulbib mees ja naeris puhub sädelevaid mulle.")
text.tag_layer( vm_analyzer.input_layers )
vm_analyzer.tag(text)
text['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Kärbes', [{'normalized_text': 'Kärbes', 'lemma': 'Kärbe', 'root': 'Kärbe', 'root_tokens': ['Kärbe'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'H'}, {'normalized_text': 'Kärbes', 'lemma': 'Kärbes', 'root': 'Kärbes', 'root_tokens': ['Kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'H'}, {'normalized_text': 'Kärbes', 'lemma': 'kärbes', 'root': 'kärbes', 'root_tokens': ['kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('hulbib', [{'normalized_text': 'hulbib', 'lemma': 'hulpima', 'root': 'hulpi', 'root_tokens': ['hulpi'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('mees', [{'normalized_text': 'mees', 'lemma': 'mees', 'root': 'mees', 'root_tokens': ['mees'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}, {'normalized_text': 'mees', 'lemma': 'mesi', 'root': 'mesi', 'root_tokens': ['mesi'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('naeris', [{'normalized_text': 'naeris', 'lemma': 'naerma', 'root': 'naer', 'root_tokens': ['naer'], 'ending': 'is', 'clitic': '', 'form': 's', 'partofspeech': 'V'}, {'normalized_text': 'naeris', 'lemma': 'naeris', 'root': 'naeris', 'root_tokens': ['naeris'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}, {'normalized_text': 'naeris', 'lemma': 'naeris', 'root': 'naeris', 'root_tokens': ['naeris'], 'ending': 's', 'clitic': '', 'form': 'sg in', 'partofspeech': 'S'}]),
Span('puhub', [{'normalized_text': 'puhub', 'lemma': 'puhuma', 'root': 'puhu', 'root_tokens': ['puhu'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('sädelevaid', [{'normalized_text': 'sädelevaid', 'lemma': 'sädelev', 'root': 'sädelev', 'root_tokens': ['sädelev'], 'ending': 'id', 'clitic': '', 'form': 'pl p', 'partofspeech': 'A'}]),
Span('mulle', [{'normalized_text': 'mulle', 'lemma': 'mull', 'root': 'mull', 'root_tokens': ['mull'], 'ending': 'e', 'clitic': '', 'form': 'pl p', 'partofspeech': 'S'}, {'normalized_text': 'mulle', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': 'lle', 'clitic': '', 'form': 'sg all', 'partofspeech': 'P'}, {'normalized_text': 'mulle', 'lemma': 'mulle', 'root': 'mulle', 'root_tokens': ['mulle'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

In [6]:
# Use BertMorphTagger to disambiguate the layer
morph_disambiguator.retag( text )
text['morph_analysis']

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Kärbes', [{'normalized_text': 'Kärbes', 'lemma': 'kärbes', 'root': 'kärbes', 'root_tokens': ['kärbes'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('hulbib', [{'normalized_text': 'hulbib', 'lemma': 'hulpima', 'root': 'hulpi', 'root_tokens': ['hulpi'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('mees', [{'normalized_text': 'mees', 'lemma': 'mees', 'root': 'mees', 'root_tokens': ['mees'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('ja', [{'normalized_text': 'ja', 'lemma': 'ja', 'root': 'ja', 'root_tokens': ['ja'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'J'}]),
Span('naeris', [{'normalized_text': 'naeris', 'lemma': 'naeris', 'root': 'naeris', 'root_tokens': ['naeris'], 'ending': '0', 'clitic': '', 'form': 'sg n', 'partofspeech': 'S'}]),
Span('puhub', [{'normalized_text': 'puhub', 'lemma': 'puhuma', 'root': 'puhu', 'root_tokens': ['puhu'], 'ending': 'b', 'clitic': '', 'form': 'b', 'partofspeech': 'V'}]),
Span('sädelevaid', [{'normalized_text': 'sädelevaid', 'lemma': 'sädelev', 'root': 'sädelev', 'root_tokens': ['sädelev'], 'ending': 'id', 'clitic': '', 'form': 'pl p', 'partofspeech': 'A'}]),
Span('mulle', [{'normalized_text': 'mulle', 'lemma': 'mina', 'root': 'mina', 'root_tokens': ['mina'], 'ending': 'lle', 'clitic': '', 'form': 'sg all', 'partofspeech': 'P'}]),
Span('.', [{'normalized_text': '.', 'lemma': '.', 'root': '.', 'root_tokens': ['.'], 'ending': '', 'clitic': '', 'form': '', 'partofspeech': 'Z'}])])

As for the limitation, the disambiguator does not resolve lemma ambiguities ( such as the ambiguity of the word _'teod'_ between lemmas _'tegu'_ and _'tigu'_ ).

---